Referance: https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/bpp.ipynb

In [1]:
%pip install -q amplpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 7.4 MB/s eta 0:00:00


In [13]:
import pandas as pd
from random import randint
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["gcg", "gurobi", "scip", "highs", "cbc"],  # modules to install
    license_uuid="default",  # license to use
)

# Problem

Given $n$ items with weights $w_{i}$ for all $i \in \{1,...,n\}$ and bins with capacity $C$ (for each bin), the bin packing problem assigns each item $i$ to a bin while minimizing the number of used bins. BPP can be modeled as follows:

$$
\begin{aligned}
\text{minimize} \quad &\sum_{j = 1}^{m} y_{j} \\
\text{subject to} \quad &\sum_{i = 1}^{n} w_{i} x_{i j} \leq C y_{j} \quad \forall j \in \{1,\ldots, m\} \\
&\sum_{j = 1}^{m} x_{i j} = 1 \quad \forall i \in \{1,\ldots,n\} \\
&x_{i j}, y_j \in \{0,1\} \quad \forall i \in \{1,\ldots,n\}, j \in \{1,\ldots,m\}
\end{aligned}
$$

# Determine a proper number of bins.

* A lower bound of $m$ can be found from $\sum w_i \leq C m$, but the inequality is already included in the first two conditions above. Moreover, what we need is an upper bound of $m$ which will be used as a parameter in the AMPL model.

* Let $W$ be the maximum weight. An upper bound of $m$ can be set by the minimum number of bins to which we assign $n$ items with all weights $W$. Each bin can have up to $r$ items, where $r = \text{floor}(C/W)$. Then, the number of bins is $\text{ceil}(n/r)$.

In [15]:
%%ampl_eval
param n;
param C;

set I = 1..n;
param w {I} > 0;
param maxVal := max {i in I} w[i];
param maxbins := ceil(n / floor(C / maxVal));

set J = 1..maxbins;

var x {I,J} binary;
var y {J} binary;

minimize Cost:
  sum {j in J} y[j];

subject to b_Capacity {j in J}:
  sum {i in I} w[i] * x[i,j] <= C * y[j];

subject to m_Allocate {i in I}:
   sum {j in J} x[i,j] = 1;

In [16]:
n = 30
C = 120
w = [randint(10, 100) for i in range(n)]

ampl.param["n"] = n
ampl.param["C"] = C
ampl.param["w"] = {i+1: w[i] for i in range(n)}

In [17]:
ampl.option["solver"] = "cbc"
ampl.option["cbc_options"] = "outlev=1"
ampl.solve()

cbc 2.10.10:   tech:outlev = 1
Welcome to the CBC MILP Solver 
Version: 2.10.10 
Build Date: Sep  5 2023 

command line - Cbc_C_Interface -log 1 -solve -quit (default strategy 1)
Continuous objective value is 16.1333 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 7 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 10 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 7 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 7 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 7 strengthened rows, 0 substitutions
Cgl0004I processed model has 60 rows, 930 columns (930 integer (930 of which binary)) and 18

In [24]:
df = pd.DataFrame([ij for ij, val in ampl.var["x"].to_dict().items() if round(val)==1])
df.columns = ['item_no', 'bin_no']
df = df.set_index('item_no').sort_index()
df

,bin_no
item_no,
1,6
2,27
3,25
4,25
5,21
6,5
7,30
8,19
9,29


In [27]:
df.groupby('bin_no').apply(lambda r: r.index.to_list())

bin_no
1         [15]
2         [26]
3     [12, 27]
4         [11]
5          [6]
6          [1]
7     [24, 28]
11    [13, 21]
15    [10, 14]
17    [16, 29]
19         [8]
21     [5, 22]
22    [19, 30]
25      [3, 4]
26    [17, 23]
27     [2, 20]
29     [9, 18]
30     [7, 25]
dtype: object

In [8]:
# ampl.option["solver"] = "gcg"
# ampl.option["gcg_options"] = "outlev=1"
# ampl.solve()

In [9]:
# ampl.option["solver"] = "gurobi"
# ampl.option["gurobi_options"] = "outlev=1"
# ampl.solve()

In [12]:
# ampl.option["solver"] = "scip"
# ampl.option["scip_options"] = "outlev=1"
# ampl.solve()

# Try
* bins have different capacities? Objective can be different, say, minimize the number of bins or minimize the total capacities of bins used...